# Forces and Hessian 

In this tutorial, we analytically calculate the force vector and Hessian matrix of benzoic acid in internal coordinates. We then verify these results by comparing them to the same quantities calculated via finite differences. 

## Background: Z-Matrix
Before we start performing calculations, let's take some time to understand the Z-matrix and how it is represented wwithin PyZMAT.

A **Z-matrix** is a way to encode a molecule’s geometry using internal coordinates—bond lengths, bond angles, and dihedral angles—instead of Cartesian $(x, y, z)$ positions. Each new atom is placed relative to one or more previously defined atoms:

1. The **first** atom is placed at the origin.  
2. The **second** atom is placed at a specified **bond length** from atom 1.  
3. The **third** atom is placed at a specified **bond length** from atom 2 and a **bond angle** relative to atoms 2–1.  
4. Subsequent atoms use a **bond length**, **bond angle**, and **dihedral angle** defined with respect to three earlier atoms.


A common way of representing the Z-matrix is:

| Atom  | Bond Ref | Bond Val   | Angle Ref | Angle Val   | Dihedral Ref | Dihedral Val  |
|-------|----------|------------|-----------|-------------|--------------|---------------|
| Atom 0 | –        | –          | –         | –           | –            | –             |
| Atom 1 | Atom 0     | $b_{1,0}$   | –         | –           | –            | –             |
| Atom 2 | Atom 0     | $b_{2,0}$   | Atom 1      | $a_{2,0,1}$    | –            | –             |
| Atom 3 | Atom 2     | $b_{3,2}$   | Atom 0      | $a_{3,2,0}$   | Atom 1         | $d_{3,2,0,1}$      |
| …     | …        | …          | …         | …           | …            | …             |
| Atom $i$     | Atom $j$        | $b_{i,j}$          |  Atom $k$        | $a_{i,j,k}$           | Atom $l$            | $d_{i,j,k,l}$             |

In PyZMAT, this is stored as two parallel lists - a list of values and a list of connectivities. For the table above,
```python
zmat = [
    [Species0, None, None, None],    #Atom 0 
    [Species1, b_10, None, None],    #Atom 1 
    [Species2, b_20, a_201, None],    #Atom 2
    [Species3, b_32, a_320, d_3201],    #Atom 3
    ...
    [Speciesi, b_ij, a_ijk, d_ijkl]    #Atom I
    ...
]
```
and
```python
zmat_conn = [
    (Species0, None, None, None),    #Atom 0
    (Species1, 0, None, None),    #Atom 1 
    (Species2, 0, 1, None),    #Atom 2
    (Species3, 2, 0, 1),    #Atom 3
    ...
    (Speciesi, j, k, l)    #Atom I
    ...
]
```

* ```SpeciesI```: Chemical symbol of Atom $i$ (string)
* ```Bndi_val, Angi_val, Dihi_val```: bond length $b_{i,j}$ [A], bond angle $a_{i,j,k}$ [deg], and dihedral angle $d_{i,j,k,l}$ [deg] of Atom $i$ defined with respect to Atom $j$, Atom $k$, and Atom $l$ (floats)
* ```j, k, l```: Indices of reference atoms Atom $i$, Atom $j$, and Atom $k$ (integers)

We will see this in practice early on in this tutorial

## 1. Setting up structure
We begin by importing all modules from PyZMAT as well as a few modules from ase.

In [1]:
from pyzmat import *
from ase import Atoms
from ase.io import read

We can load our structure from an .xyz file and an initial set of Z-matrix connectivities defined with respect to the line numbers in .xyz file.

In [2]:
xyz_file = 'BENZAC02.xyz'
zmat_def = [
    (2, None, None, None),
    (3, 2, None, None),
    (4, 3, 2, None),
    (5, 4, 3, 2),
    (6, 5, 4, 3),
    (7, 6, 5, 4),
    (8, 7, 6, 5),
    (1, 2, 3, 4),
    (0, 2, 3, 1),
    (14, 0, 2, 3),
    (9, 4, 3, 8),
    (10, 5, 4, 3),
    (11, 6, 5, 4),
    (12, 7, 6, 5),
    (13, 8, 7, 6),
]
atoms = read(xyz_file)

With this set of information, the function atoms_2_zmat_init in the ZmatUtils class allows us to form a re-ordered Z-matrix based on our initial connectivity definition. The aforementioned zmat and zmat_conn are then consistent with this new ordering. Note that zmat_conn is distinct from zmat_def:

In [3]:
zmat, zmat_conn = ZmatUtils.atoms_2_zmat_init(atoms, zmat_def)
print(zmat_conn)

[('C', None, None, None), ('C', 0, None, None), ('C', 1, 0, None), ('C', 2, 1, 0), ('C', 3, 2, 1), ('C', 4, 3, 2), ('C', 5, 4, 3), ('O', 0, 1, 2), ('O', 0, 1, 7), ('H', 8, 0, 1), ('H', 2, 1, 6), ('H', 3, 2, 1), ('H', 4, 3, 2), ('H', 5, 4, 3), ('H', 6, 5, 4)]


With zmat and zmat_conn defined, we are now ready to form a ZMatrix object that represents our molecular structure. Let's visualise this:

In [4]:
benzac = ZMatrix(zmat, zmat_conn)
benzac.view_ase()

In [5]:
benzac.attach_calculator('mace')

/root/miniconda3/envs/pyzmat_test/lib/python3.13/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.load

Using MACE-OFF23 MODEL for MACECalculator with /root/.cache/mace/MACE-OFF23_large.model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/root/miniconda3/envs/pyzmat_test/lib/python3.13/site-packages/mace/calculators/mace.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_locat

## 2. Calculating forces

The Cartesian force on degree of freedom $i$ is given by the negative gradient of the potential:
$$F^\mathrm{cart}_i \;=\;-\,\frac{\partial U}{\partial r_i}$$
where $U$ is the system’s potential energy and $r_i$ is the $i$-th Cartesian coordinate.

The internal coordinates equivalent to this is 
$$F^\mathrm{int}_s \;=\;-\,\frac{\partial U}{\partial \theta_s}$$
where $\theta_s$ is the $s$-th internal coordinate.

Via the chain rule, one can write
$$F^\mathrm{int}_s = \sum_i^{3N} \frac{\partial r_i}{\partial\theta_s}\frac{\partial U}{\partial r_i}$$
or in matrix form
$$\mathbf{F}^\mathrm{int}=\mathbf{B}\cdot\mathbf{F}^\mathrm{cart}$$
where $\mathbf{B}$ is the Wilson-$\mathbf{B}$ matrix of $B_{i,s}=(\partial r_i/\partial\theta_s)_{i,s}$
Given this and that we have analytical forces in Cartesian coordinates from ASE, we have two strategies for computing forces in internal coordinates:
1. **Computing $\mathbf{B}$ by finite difference**: perturb entry corresponding to $\theta_s$ in ```zmat``` in forward and backward directions, build cartesian molecule for both, and find the gradient on $r_i$
2. Computing $\mathbf{B}$ analytically

Both approaches are implemented in PyZMAT. The first approach: 

In [15]:
forces_fd = benzac.get_fd_forces()
PrintUtils.print_forces(forces_fd, benzac.zmat)

C1                                                                     
C2     -0.011089769934236                                              
C3      0.024308308929303      0.214460595682476                       
C4      0.503049763325678      0.067817546979841     -0.019823275403291
C5      1.341866889510375      0.531738087767389     -0.059131135368434
C6      1.032219693366688      0.802688551679668      0.046771470354587
C7      0.446196040384410      0.569322920284653     -0.067724274125442
O1     -3.324666460110532      0.644216519573160     -0.004360009331533
O2      3.334342765418469     -0.414158532828518      0.000000759638751
H1      6.844012591957627     -0.869443960036592     -0.006858868557243
H2      5.900015345853051     -0.037674825614649     -0.011703283261095
H3     10.088225688438163      0.443512753582513      0.170222479189539
H4      5.665570519199710      0.021002719943851     -0.032410867471460
H5      8.709121331166092      0.201329753833941      0.17478845

The second approach:

In [16]:
forces = benzac.get_forces()
PrintUtils.print_forces(forces, benzac.zmat)

C1                                                                     
C2     -0.011089769768579                                              
C3      0.024308309289766      0.214460595581994                       
C4      0.503049763476398      0.067817546980054     -0.019823275293359
C5      1.341866889317566      0.531738087939980     -0.059131135434204
C6      1.032219693353964      0.802688551807573      0.046771470388732
C7      0.446196040169674      0.569322920261752     -0.067724274134237
O1     -3.324666460111706      0.644216519596916     -0.004360009331607
O2      3.334342765329354     -0.414158532772614      0.000000759639989
H1      6.844012591962354     -0.869443960110630     -0.006858868536359
H2      5.900015345853305     -0.037674825592923     -0.011703283221664
H3     10.088225688499818      0.443512753599915      0.170222479247080
H4      5.665570519217803      0.021002720028290     -0.032410867473081
H5      8.709121331315673      0.201329753809826      0.17478845

At a glance, the two methods yield consistent forces. For a more rigorous check, we can use ```np.allclose``` with an absolute tolerance of $10^{-7}$:

In [18]:
import numpy as np

if np.allclose(forces, forces_fd, atol = 1e-7):
    print("The arrays are equal.")
else:
    print("The arrays differ beyond the allowed tolerance.")

The arrays are equal.


### Exercise

For further validation, implement a method to compute $-\partial U/\partial \theta_s$ via finite difference and compare the results. For each gradient:
1. Perturb $\theta_s$ in the forward direction by $\Delta\theta_s$
2. Generate ```atoms``` with ```zmat_2_atoms(zmat, zmat_conn)```
3. Calculate forward energy with ASE
4. Repeat for a backward perturbation
5. Calculate gradient $-\partial U/\partial \theta_s = -(U^\mathrm{fwd}-U^\mathrm{bwd})/2\Delta\theta_s$

## 2. Calculating Hessian

The Hessian matrix is the second derivative of the potential with respect to atomic positions. In cartesian coordinates,
$$
H^\mathrm{cart}_{i,i'} \;=\; \frac{\partial U}{\partial r_{i'} \partial r_i}
$$
In internal coordinates,
$$
H^\mathrm{int}_{s,s'} \;=\; \frac{\partial U}{\partial \theta_{s'} \partial \theta_s}
$$

By applying the chain rule again, we eventually arrive at

$$
\begin{aligned}
H^\mathrm{int}_{s,s'}
&=
\sum_{i=1}^{3N}
  \frac{\partial r_i}{\partial \theta_{s'} \partial \theta_s}
  \frac{\partial U}{\partial r_i}
\;+\;
\bigl(\mathbf{B}\cdot\mathbf{H}^\mathrm{cart}\cdot\mathbf{B}^\mathrm{T}\bigr)_{s,s'}.
\end{aligned}
$$
